In [1]:
import torch
import numpy as np
import sys
sys.path.append('/burg/home/jb5005')
from struct_diff.data.datamodule import ContinuousStructTokenDataModule
from struct_diff.data.dataset import ContinuousStructTokenDataset
from esm.pretrained import ESM3_structure_encoder_v0, ESM3_structure_decoder_v0
import time


In [2]:
datamodule = ContinuousStructTokenDataModule(base_path = '/pmglocal/jb5005/struct_diff_data/data', 
                                             df_path = '/burg/home/jb5005/struct_diff/data/seq_len.csv',
                                             num_workers = 0,
                                             decoder = None)
datamodule.setup('train')

Fetching 22 files:   0%|          | 0/22 [00:00<?, ?it/s]

/pmglocal/jb5005/mambaforge/envs/struct_diff/lib/python3.12/site-packages/esm/pretrained.py:43: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(


In [5]:
dataloader = datamodule.get_dataloader()

In [4]:
first_element = next(iter(dataloader))

/burg/home/jb5005/struct_diff/data/dataset.py:57: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(cont_emb)


In [4]:
dataset = ContinuousStructTokenDataset(base_path = '/pmglocal/jb5005/struct_diff_data/data', df_path = '/burg/home/jb5005/struct_diff/data/seq_len.csv')

In [8]:
dataset.__getitem__(18).shape

Time to load pdb: 0.0007641315460205078
Time to create protein object: 5.0067901611328125e-06
Time to convert to continuous embeddings: 0.0003159046173095703


torch.Size([104, 1280])

In [17]:
decoder = ESM3_structure_decoder_v0(device='cuda')

In [21]:
start = time.time()
arr = np.arange(0,1000)
print(decoder.embed(torch.tensor(arr, device = "cuda")).shape)
print(f"Time taken for embedding: {time.time() - start}")

torch.Size([1000, 1280])
Time taken for embedding: 0.0008194446563720703


In [6]:
first_element.shape

torch.Size([32, 299, 1280])

In [10]:
first_element[0,:,:]

tensor([[-0.0233,  0.1030,  0.0322,  ...,  0.0559,  0.0364, -0.0383],
        [-0.0035, -0.0688, -0.0752,  ..., -0.0752, -0.0840,  0.0508],
        [-0.1943, -0.0403,  0.0334,  ...,  0.0073, -0.1240, -0.0508],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       device='cuda:0')